# Generic optimization problem

This notebook is performing a 3 bit minimization for the following problem 

$$
x*y - x*z + 2*y*z + x - 2*y - 3*z
$$

## **Step 0**: Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import RealAmplitudes

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# quadratic_program
from quadratic_program import QuadraticProgram

# Docplex - classical description of optimization problems
from docplex.mp.model import Model

# translations
from translators import docplex_mp_to_qp
from translators import qubo_to_sparse_pauli_op

# workflows 
from workflows import QuadraticProgramPostprocess, QuadraticProgramConverter

# SPSA
from spsa import minimize_spsa

# Quantum Middleware
import os
from quantum_serverless import Provider, distribute_program, distribute_task

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


## Load IBM Quantum Middleware

In [2]:
provider = Provider(
#    token="YOUR_TOKEN_FROM_STAGING",
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000")
)
provider

<Provider: gateway-provider>

## **Step 1** Map the problem to a Quantum Native format (Set of Operators, and a set of Quantum Circuits)

Specify optimization problem using docplex and convert to Quadratic Unconstained Binary Opimization (QUBO) problem that can be cast as an Ising Hamiltonian suitable for a quantum solution.

In [3]:
def build_model():
    mdl = Model("docplex model")
    x = mdl.binary_var("x")
    y = mdl.binary_var("y")
    z = mdl.binary_var("z")
    mdl.minimize(x*y - x*z + 2*y*z + x - 2*y - 3*z)
    return mdl

### Convert to our `QuadraticProgram` format

In [4]:
def build_quadratic_program(mdl):
    qp = docplex_mp_to_qp(mdl)
    return qp

### Classical transformation to QUBO problem and Ising Hamiltonian

In [5]:
def build_quadratic_transformer(qp):
    quadratic_transformer = QuadraticProgramConverter()
    qubo = quadratic_transformer.run(qp)
    hamiltonian, offset = qubo_to_sparse_pauli_op(qubo)
    return quadratic_transformer, qubo, hamiltonian

### Define ansatz circuit from circuit library

In [6]:
def quantum_solution_setup(hamiltonian):
    ansatz = RealAmplitudes(hamiltonian.num_qubits, entanglement = 'linear', reps=2)
    return ansatz

## **Step 2**: Optimize the circuits and the operators to be measured

In [7]:
from qiskit.compiler import transpile

def optimize_circuits(ansatz):
    ansatz_ibm = transpile(ansatz, basis_gates = ['cz', 'sx', 'rz'],  coupling_map =[[0, 1], [1, 2]], optimization_level=3)
    return ansatz_ibm

In [8]:
from permute_sparse_pauli_op import permute_sparse_pauli_op

def optimize_operators(ansatz, ansatz_ibm, hamiltonian):
    layout = ansatz_ibm.layout.initial_layout
    hamiltonian_ibm = permute_sparse_pauli_op(hamiltonian,layout, ansatz.qubits)
    return hamiltonian_ibm

## **Step 3**: Execute using a quantum primitive function (estimator or sampler)

### Standard cost function definition

In [9]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [10]:
def setup_estimator_sampler():
    #session = Session(backend=backend)
    #estimator = Estimator(session=session, options={"shots": int(1e4)})
    #sampler = Sampler(session=session, options={"shots": int(1e4)})
    estimator = QiskitEstimator(options={"shots": int(1e4)})
    sampler = QiskitSampler(options={"shots": int(1e4)})
    return estimator, sampler

### Perform minimization using classical SPSA optimizer

In [11]:
def minimize(ansatz, ansatz_ibm, hamiltonian_ibm, estimator):
    x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
    res = minimize_spsa(cost_func, x0, args=(ansatz_ibm, hamiltonian_ibm, estimator), maxiter=100)
    return res

### Computute sampled distribution at optimal values

In [12]:
def compute_distribution(ansatz_ibm, sampler, res):
    # Assign solution parameters to ansatz
    qc = ansatz_ibm.assign_parameters(res.x)
    qc.measure_all()
    samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
    return samp_dist

## **Step 4**: Post-processing of the results to return either a plot or the answer
Transform quantum solution and convert back into classical variable space

In [13]:
def post_processing(qubo, quadratic_transformer, samp_dist):
    solution = QuadraticProgramPostprocess(qubo, quadratic_transformer).run(samp_dist)
    return solution

### Setup Middleware

In [14]:
@distribute_program(provider, working_dir="./", dependencies=["docplex"])
def program_optimization():
    
    # Step 1
    
    mdl = build_model()
    print(mdl.export_as_lp_string())
    
    qp = build_quadratic_program(mdl)
    print(qp.prettyprint())
    
    quadratic_transformer, qubo, hamiltonian = build_quadratic_transformer(qp)
    print(qubo.prettyprint())
    print(hamiltonian)
    
    ansatz = quantum_solution_setup(hamiltonian)
    print(ansatz.decompose())
    
    # Step 2
    
    ansatz_ibm = optimize_circuits(ansatz)
    print(ansatz_ibm)
    
    hamiltonian_ibm = optimize_operators(ansatz, ansatz_ibm, hamiltonian)
    print(hamiltonian_ibm)
    
    # Step 3
    
    estimator, sampler = setup_estimator_sampler()
    res = minimize(ansatz, ansatz_ibm, hamiltonian_ibm, estimator)
    print(res)
    
    samp_dist = compute_distribution(ansatz_ibm, sampler, res)
    solution = post_processing(qubo, quadratic_transformer, samp_dist)
        
    return solution.tolist()

In [15]:
job = program_optimization()
job

<Job | dc715e62-60d3-4498-98e7-13e94236357b>

In [17]:
job.status()

'SUCCEEDED'

In [18]:
job.result()

'[0.0, 0.0, 1.0]'

In [19]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x - 2 y - 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y - 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y - 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[-0.5 +0.j,  0.25+0.j,  1.25+0.j,  0.25+0.j, -0.25+0.j,  0.5 +0.j])
     ┌──────────┐     ┌──────────┐                 ┌──────────┐            
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[3]) ├──────────────■──┤ Ry(θ[6]) ├────────────
     ├──────────┤┌